In [1]:
#data Preparation

import pandas as pd
from preprocess import preprocess_text

#loading the data
train_set = pd.read_csv('semEval dataest/semeval-2017-train.csv', delimiter='\t')
test_set = pd.read_csv('semEval dataest/semeval-2017-test.csv', delimiter='\t')
dev_set = pd.read_csv('semEval dataest/semeval-2017-dev.csv', delimiter='\t')

#preprocessing the data
train_set['text'] = train_set['text'].apply(preprocess_text)
train_set.head()

,label,text
0,1,walking chelsea time day rather lovely love lo...
1,0,first play night aaron rodgers int udfa cb bra...
2,0,drove bike today miles felt like jim carrey irene
3,-1,looking temp outside hpw get hotter sun goes f...
4,0,rt redarmy therefore still expect arsenal sign...


In [ ]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

# Loading  BERT model architecture
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Loading BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example
input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, add_special_tokens=True)
inputs = tf.constant([input_ids])

# Get BERT model outputs
outputs = bert_model(inputs)
outputs


C:\Users\lenovo\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lenovo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
from transformers import BertForSequenceClassification
import torch
import numpy as np
import pandas as pd


#Training the teacher model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenize input data
def tokenize_data(data, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for text in data['text']:
        encoded_dict = tokenizer.encode_plus(text )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

max_seq_length = 128
num_labels = len(data['label'].unique())

train_input_ids, train_attention_masks = tokenize_data(train_data, tokenizer, max_seq_length)
val_input_ids, val_attention_masks = tokenize_data(val_data, tokenizer, max_seq_length)

train_labels = torch.tensor(train_data['label'])
val_labels = torch.tensor(val_data['label'])

# Create DataLoader
batch_size = 32

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and training loop
optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 5
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    val_accuracy = 0
    val_loss = 0
    for batch in val_dataloader:
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        with torch.no_grad():
            outputs = model(**inputs)
            val_loss += outputs.loss.item()
            logits = outputs.logits
            preds = np.argmax(logits, axis=1)
            val_accuracy += np.sum(preds == batch[2].numpy())

    val_loss /= len(val_dataloader)
    val_accuracy /= len(dev_set)

    print(f'Epoch {epoch+1}/{epochs}')
    print(f'Val Loss: {val_loss:.4f} Val Acc: {val_accuracy:.4f}')